In [1]:
import sqlite3
import os

#First make sure if the database is already there that we erase it prior to this lesson
if os.path.exists("Basics.db"):
    os.remove("Basics.db")
#Import sqlite3 which will allow us to create a SQL database and also 
conn = sqlite3.connect('Basics.db')
#We create a connection to a sqlite database called Basics.db
#If this is not yet created then it gets created automatically

In [2]:
#The first step in working with this is creating a cursor
#The cursor will let us execute SQL commands
cur = conn.cursor()

In [3]:
#We use c.execute() to execute things in SQL. Let's create a table of students and grades
#Create table makes our table followed by the name of the table and in parantheses the column names and types
#The variables are name and grade, and there tpe is varchar (similar to a string) with 100 characters and an integer
query = """CREATE TABLE students(
name varchar(100),
grade int
);"""
cur.execute(query)

In [4]:
#What happens if we try and make the same table twice?
cur.execute(query)

OperationalError: table students already exists

In [5]:
#You can delete a table with drop followed by the name
#And then add it back
query = "DROP TABLE students;"
cur.execute(query)
query = """CREATE TABLE students(
name varchar(100),
grade int
);"""
cur.execute(query)

In [6]:
#You can insert into a SQL database with INSERT INTO followed by the table and values
#Notice the quotes around Tom because it is a character
query = """INSERT INTO students
VALUES ("Tom",90);"""
cur.execute(query)

In [7]:
#You can also selectively insert into only certain columns
query = """INSERT INTO students (name)
VALUES ("Josh");"""
cur.execute(query)

In [8]:
#You can also insert multiple rows at the same time
query = """INSERT INTO students
VALUES ("Mary",85),("Clair",95);"""
cur.execute(query)

In [9]:
#Use commit() to save your changes
conn.commit()

In [10]:
#Then close the connection with close()
conn.close()

In [11]:
#Let's connect again
conn = sqlite3.connect('Basics.db')
cur = conn.cursor()

In [12]:
#The way we view all data in a database is by using SELECT * FROM followed by our table name
query = """SELECT * FROM students;"""
cur.execute(query)

In [13]:
#But we do not actually see the result, to do that we iterate over the return
for row in cur.execute('SELECT * FROM students;'):
    print(row)

('Tom', 90)
('Josh', None)
('Mary', 85)
('Clair', 95)


In [14]:
#We can also select only one or more column by giving columns instead of *
for row in cur.execute('SELECT name FROM students;'):
    print(row)

('Tom',)
('Josh',)
('Mary',)
('Clair',)


In [15]:
#You can also select rows based on a condition
for row in cur.execute('SELECT * FROM students WHERE grade>87;'):
    print(row)

('Tom', 90)
('Clair', 95)


In [16]:
#If we want only names we can combine the last two queries
for row in cur.execute('SELECT name FROM students WHERE grade>87;'):
    print(row)

('Tom',)
('Clair',)


In [17]:
#There are also clauses we can use such as AND/OR
for row in cur.execute('SELECT name FROM students WHERE grade>87 AND grade<93;'):
    print(row)
print()
print()
for row in cur.execute('SELECT name FROM students WHERE grade<87 OR grade>93;'):
    print(row)

('Tom',)


('Mary',)
('Clair',)


In [18]:
#If you use NOT you can reverse logic
#But if you have a null value it will not be included in either
for row in cur.execute('SELECT name FROM students WHERE grade>87 AND grade<93;'):
    print(row)
print()
print()
for row in cur.execute('SELECT name FROM students WHERE NOT (grade>87 AND grade<93);'):
    print(row)

('Tom',)


('Mary',)
('Clair',)


In [19]:
#To find NULL records....
for row in cur.execute('SELECT name FROM students WHERE grade IS NULL;'):
    print(row)

print()
print()
#And non-null
for row in cur.execute('SELECT name FROM students WHERE grade IS NOT NULL;'):
    print(row)

('Josh',)


('Tom',)
('Mary',)
('Clair',)


In [20]:
#We can add a column by using alter
query = """ALTER TABLE students
ADD COLUMN ID int;"""
cur.execute(query)

In [21]:
for row in cur.execute('SELECT * FROM students;'):
    print(row)

('Tom', 90, None)
('Josh', None, None)
('Mary', 85, None)
('Clair', 95, None)


In [22]:
#You can give values with update
query = """UPDATE students SET ID = 1;"""
cur.execute(query)
for row in cur.execute('SELECT * FROM students;'):
    print(row)

('Tom', 90, 1)
('Josh', None, 1)
('Mary', 85, 1)
('Clair', 95, 1)


In [23]:
#You can add in a clause
query = """UPDATE students SET ID = 5 WHERE grade <92;"""
cur.execute(query)
for row in cur.execute('SELECT * FROM students;'):
    print(row)

('Tom', 90, 5)
('Josh', None, 1)
('Mary', 85, 5)
('Clair', 95, 1)


In [24]:
#Using fetchall is a quick way to grab all results
cur.execute('SELECT * FROM students;')
data = cur.fetchall()
print(data)

[('Tom', 90, 5), ('Josh', None, 1), ('Mary', 85, 5), ('Clair', 95, 1)]


In [25]:
#DISTINCT allows for the unique values
cur.execute('SELECT DISTINCT ID FROM students;')
data = cur.fetchall()
print(data)

[(5,), (1,)]


In [26]:
#Let's duplicate a row
query = """INSERT INTO students
VALUES ('Tom', 90, 5);"""
cur.execute(query)

In [27]:
#With select * we get all the records
cur.execute('SELECT * FROM students;')
data = cur.fetchall()
print(data)

#With select DISTINCT * we get rid of any duplicates where all the columns match
cur.execute('SELECT DISTINCT * FROM students;')
data = cur.fetchall()
print(data)

[('Tom', 90, 5), ('Josh', None, 1), ('Mary', 85, 5), ('Clair', 95, 1), ('Tom', 90, 5)]
[('Tom', 90, 5), ('Josh', None, 1), ('Mary', 85, 5), ('Clair', 95, 1)]


In [28]:
#Let's duplicate a row except with grades different
query = """INSERT INTO students
VALUES ('Tom', 85, 5);"""
cur.execute(query)

In [29]:
#Notice how we can if we look for distinct of name + ID only versus all then we get different results
cur.execute('SELECT DISTINCT * FROM students;')
data = cur.fetchall()
print(data)

cur.execute('SELECT DISTINCT name, ID FROM students;')
data = cur.fetchall()
print(data)

[('Tom', 90, 5), ('Josh', None, 1), ('Mary', 85, 5), ('Clair', 95, 1), ('Tom', 85, 5)]
[('Tom', 5), ('Josh', 1), ('Mary', 5), ('Clair', 1)]


In [30]:
#You can also delete specific records with DELETE
cur.execute('DELETE FROM students WHERE name=="Tom";')
cur.execute('SELECT * FROM students;')
data = cur.fetchall()
print(data)

[('Josh', None, 1), ('Mary', 85, 5), ('Clair', 95, 1)]
